In [ ]:
#!pip install setfit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from datasets import Dataset
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from sklearn.metrics import classification_report
import json

def write_json(output_path, json_data):
    with open(output_path, "w", encoding="utf-8") as outfile:
        json.dump(json_data, outfile, indent=4, ensure_ascii=False)


In [ ]:
root_drive_dir = '/content/drive/My Drive/Colab Notebooks/DataHub Projects/CrispyWork/edos/data/'

dataset = pd.read_csv(os.path.join(root_drive_dir, "edos_labelled_aggregated.csv"))
dataset = dataset[dataset['label_sexist'] == 'sexist'][["text", "label_category", "split"]]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### train, dev, test split

In [ ]:
train = dataset[dataset['split'] == 'train']
train = train.reset_index(drop=True)
print('Train set shape:', train.shape)

dev = dataset[dataset['split'] == 'dev']
dev = dev.reset_index(drop=True)
print('Validation set shape:', dev.shape)

test = dataset[dataset['split'] == 'test']
test = test.reset_index(drop=True)
print('Test set shape:', test.shape)

Train set shape: (3398, 3)
Validation set shape: (486, 3)
Test set shape: (970, 3)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### pick n random samples per class

In [ ]:
random_sample_no = 40
seed = 444

threats_samples = train[train['label_category'] == '1. threats, plans to harm and incitement'].sample(n=random_sample_no, random_state=seed)
derogation_samples = train[train['label_category'] == '2. derogation'].sample(n=random_sample_no, random_state=seed)
animosity_samples = train[train['label_category'] == '3. animosity'].sample(n=random_sample_no, random_state=seed)
prejudiced_samples = train[train['label_category'] == '4. prejudiced discussions'].sample(n=random_sample_no, random_state=seed)

train_samples = pd.concat([threats_samples, derogation_samples, animosity_samples, prejudiced_samples])
train_samples = train_samples.reset_index(drop=True)

In [ ]:
# convert to Dataset format
train_df = Dataset.from_pandas(train_samples[['text', 'label_category']])
dev_df = Dataset.from_pandas(dev[['text', 'label_category']])
test_df = Dataset.from_pandas(test[['text', 'label_category']])

print(train_df)
print(dev_df)
print(test_df)

Dataset({
    features: ['text', 'label_category'],
    num_rows: 160
})
Dataset({
    features: ['text', 'label_category'],
    num_rows: 486
})
Dataset({
    features: ['text', 'label_category'],
    num_rows: 970
})


In [ ]:
sbert_path = "cross-encoder/nli-deberta-base"
labels = ["1. threats, plans to harm and incitement", "2. derogation", "3. animosity", "4. prejudiced discussions"]
output_dir = 'nli-deberta-base'
batch_size = 16
num_epochs = 5
num_epochs_clf = 15

model = SetFitModel.from_pretrained(sbert_path, labels=labels)

args = TrainingArguments(
            output_dir=output_dir,
            batch_size=batch_size,
            num_epochs=num_epochs,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
        )

trainer = Trainer(
          model=model,
          args=args,
          train_dataset=train_df,
          column_mapping={"text": "text", "label_category": "label"} ,
          metric='accuracy'
  )

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/975 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to the training dataset


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 19200
  Batch size = 16
  Num epochs = 5
  Total optimization steps = 6000


Epoch,Training Loss,Validation Loss


In [ ]:
metric = trainer.evaluate(test_df)

print("EVAL result in test:", metric)

Applying column mapping to the evaluation dataset
***** Running evaluation *****


EVAL result in test: {'accuracy': 0.4216494845360825}


In [ ]:
#finetuning on whole train
model.fit(train['text'].tolist(), train['label_category'].tolist(), num_epochs=num_epochs_clf)

dev_predictions = model.predict(dev['text'].tolist())
test_predictions = model.predict(test['text'].tolist())

dev_result = classification_report(dev['label_category'], list(dev_predictions), output_dict=True)
test_result = classification_report(test['label_category'], list(test_predictions), output_dict=True)

path_to_save_dev = "TaskB_setfit_dev.json"
path_to_save_test = "TaskB_setfit_test.json"

write_json(output_path=path_to_save_dev,
           json_data={"outputs":list(dev_result), "gold": dev['label_category'].tolist(), "evaluation": dev_result})

write_json(output_path=path_to_save_test,
           json_data={"outputs":list(test_result), "gold": test['label_category'].tolist(), "evaluation": test_result})

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
